In [1]:
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
import torch
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({
    "pad_token": "[PAD]",
    "unk_token": "[UNK]",
    "bos_token": "[BOS]",
    "eos_token": "[EOS]",
})

/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:105: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


4

In [ ]:
from datasets import load_dataset

In [19]:
dataset = load_dataset("ashaba1in/small_openwebtext")

def tokenize_function(examples):
    tokenized = tokenizer(
        examples['text'],
        padding="max_length",
        truncation=True,
        max_length=256,  
        return_tensors="pt"
    )
    tokenized['labels'] = tokenized['input_ids'].clone()
    return tokenized

tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=['text']
)

tokenized_dataset = tokenized_dataset.with_format('torch')

batch_size = 32

dataloader_egich = DataLoader(
    tokenized_dataset['train'],
    batch_size=batch_size,
    shuffle=True
)


In [6]:
import sys
import os

sys.path.append(os.path.abspath("../model"))

In [7]:
from model import GPT

In [5]:
device ='cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [5]:
vocab_size = len(tokenizer)
n_embd = 512
n_head = 8
block_size = 256
n_layer = 16
dropout = 0.3
device ='cuda' if torch.cuda.is_available() else 'cpu'
gpt = GPT(
        vocab_size=vocab_size,
        n_embd=n_embd,
        n_head=n_head,
        block_size=block_size,
        n_layer=n_layer,
        dropout=dropout,
        device=device
    ).to(device)
print(sum(p.numel() for p in gpt.parameters())/1e6, 'M parameters')

101.915733 M parameters


In [6]:
num_epochs = 2

In [7]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter(log_dir="runs/gpt_experiment_1")


2025-04-23 06:15:30.707405: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-23 06:15:30.748183: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-23 06:15:31.554086: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
import torch.optim as optim
from tqdm import tqdm
from torch.optim.lr_scheduler import SequentialLR, LambdaLR, CosineAnnealingLR


num_epochs = 1
learning_rate = 3e-4
grad_clip = 1.0
save_every = 250
print_every = 50

optimizer = optim.AdamW(gpt.parameters(), lr=learning_rate, weight_decay=0.01)
scaler = torch.cuda.amp.GradScaler(enabled=device == 'cuda')
warmup_steps = 5000 

# Планировщик с warmup + decay
# scheduler = SequentialLR(
#     optimizer,
#     schedulers=[
#         LambdaLR(optimizer, lambda step: (step + 1) / warmup_steps), 
#         CosineAnnealingLR(optimizer, T_max=5000*2)
#     ],
#     milestones=[warmup_steps]
# )
scheduler = CosineAnnealingLR(optimizer, T_max=20000)  
scaler = torch.cuda.amp.GradScaler(
    enabled=(device == device),
    init_scale=2**16,
    growth_interval=2000
)


/tmp/ipykernel_6561/2172831227.py:15: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=device == 'cuda')
/tmp/ipykernel_6561/2172831227.py:30: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(


In [ ]:
global_step = 0
best_loss = float('inf')
flag1 = False
flag2 = False

losses = []

for epoch in range(num_epochs):
    gpt.train()
    total_loss = 0.0
    progress_bar = tqdm(dataloader_egich, desc=f"Epoch {epoch+1}/{num_epochs}", leave=True)

    for batch in progress_bar:
        optimizer.zero_grad(set_to_none=True)

        inputs = batch['input_ids'].squeeze(1).to(device)
        targets = inputs.clone()[:, 1:].contiguous()

        with torch.amp.autocast(device_type='cuda', enabled=device == 'cuda'):
            logits, loss = gpt(inputs[:, :-1], targets)

        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(gpt.parameters(), grad_clip)
        
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()

        current_lr = optimizer.param_groups[0]['lr']
        writer.add_scalar("Train/Loss", loss.item(), global_step)
        writer.add_scalar("Train/Learning Rate", current_lr, global_step)
        
        with torch.no_grad():
            perplexity = torch.exp(loss.detach()).item()
        writer.add_scalar("Train/Perplexity", perplexity, global_step)

        total_loss += loss.item()
        losses.append(loss.item())
        

        if global_step % print_every == 0:
            avg_loss = total_loss / (print_every if global_step > 0 else 1)
            print(
                f"\nStep {global_step} | "
                f"Loss: {avg_loss:.4f} | "
                f"Perplexity: {perplexity:.2f} | "
                f"LR: {current_lr:.2e} | "
            )
            total_loss = 0.0
            
        if global_step % save_every == 0 and global_step > 0:
            checkpoint = {
                'model_state_dict': gpt.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'global_step': global_step,
                'loss': loss.item(),
            }

            if loss.item() < best_loss:
                best_loss = loss.item()
                torch.save(gpt.state_dict(), "gpt_best.pt")

        global_step += 1
        
        if global_step >= 20000:
            flag1 = True
            break
            
    if flag1:
        break
            

torch.save(gpt.state_dict(), "gpt_final.pt")
print("Training completed!")

Epoch 1/1:   0%|          | 1/31250 [00:00<4:42:21,  1.84it/s]


Step 0 | Loss: 4.1400 | Perplexity: 62.81 | LR: 3.00e-04 | 


Epoch 1/1:   0%|          | 51/31250 [00:23<4:01:39,  2.15it/s]


Step 50 | Loss: 4.2903 | Perplexity: 72.94 | LR: 3.00e-04 | 


Epoch 1/1:   0%|          | 101/31250 [00:46<3:58:54,  2.17it/s]

...


Step 19900 | Loss: 4.1735 | Perplexity: 63.60 | LR: 1.81e-08 | 


Epoch 1/1:  64%|██████▍   | 19951/31250 [2:37:57<1:28:15,  2.13it/s]


Step 19950 | Loss: 4.1818 | Perplexity: 59.73 | LR: 4.44e-09 | 


Epoch 1/1:  64%|██████▍   | 19999/31250 [2:38:20<1:29:04,  2.10it/s]


Training completed!


In [58]:
def generate_text(text, max_new_tokens=50):
    input_tokens = tokenizer.encode(text, return_tensors="pt").to(gpt.device)
    generated_tokens = gpt.generate(input_tokens, max_new_tokens=max_new_tokens)
    generated_text = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)
    
    return generated_text

In [33]:
input_text = "Barack Obama is"

print(generate_text(input_text))

Barack Obama is technically one of the least moral reasons the country remains idling of a U.S. pledge to tax new taxes and answer elections, sources loose in the way of forming the U.S. government-of-state party told ABC 22 less than


In [39]:
input_text_2 = "Why is the sky blue?"

print(generate_text(input_text_2))

Why is the sky blue? Take the above. I shared about the myriad authorials who offered complex and history books, ranging from armour to hair to in tones to styles.

If you think the sun was grinding to select, you can decided to look after 1967.


In [41]:
input_text_3 = "My favorite songs is"

print(generate_text(input_text_3))

My favorite songs is survival values and everything, behavior. Actually, most of the rest of the song that played playoffs and recorded – some when most of the die of millennials gave it in danger swirling and nerb about it and took the same difficulty on closets.


In [59]:
input_text_4 = "President"

print(generate_text(input_text_4, max_new_tokens=200))

President John E. Cornispy has criticised President Donald Trump for promising to impose an intelligence plan entirely. "The president asked us on it when the secretary of state got our stuff comes out and seriously the dog died. This decision is the only step I hear from him".

Trump stands for James Khan, pointing out the possibility of placing a collective view of American President Donald Trump, with Netanyahu pressing for the public's dreaming of doing what is likely to be even future sources of cooperation over his handling of climate change last year. The group hasn't repeatedly collaborated for a single rebuke of former president John J. Trump: former president Donald Trump. It looks about speaks to Shakespeare at the 1/11 summit where Trump will host a meeting of rival Republican Governor Sergei Carney. Trump's first official foreign policy announcement will be a key short-term reminder that Trump hasn't existed for the last couple months, and that by Monday the president had 

In [60]:
input_text_5 = "Sunrise is very pretty"

print(generate_text(input_text_5, max_new_tokens=200))

Sunrise is very pretty good. Snow is anything new to be certain. It’s best for science fiction tale

No, let’s not warp fire

The horror rise of scenes from UFO colonization

The world’s only impacted planets somehow action most of the seeping planets operating, finding a bigger proportion than Earth’s most important event.

Follow Yastala Kuba for the epic, public and industry-bending and high-school-educated Firefly project to build Oct. 25

Mutibont-rich, current Artists of Hiroshima

The tricky beauty is to modernize how a planet is on the planet’s farm, whether or not to live up to 30 million souls (including a fly bridge driving). It depends on its weight, its weight, and its possibility that it keeps itself is overy to its peg. The attraction of Muhammad the Fenigoral and Namoria goaling being poised to do well in the next few


In [64]:
input_text_6 = "Crocodiles are very dangerous"

print(generate_text(input_text_6, max_new_tokens=200))

Crocodiles are very dangerous when scientists continue adjusting to psilendment of Assyrification compromises, forensics.

The public arm of one of the most unpy mirrors is the fatal prefrontal cortex. Bions flux worsukes are typical only to test the functioning protection, and rates of height cannot survive or yield few light on-nule workouts with a thin response light.

The study, published in the March paper, cites a single microx. 1 microbes with less life- less warm tissue particles (currently acupuncture- cancer cells) that produce more flexibility or unpredictable behaviors. As these notes are amplified at two important processes, one could discover the sort of ability so the new sensor will cause an end to a "bitcoin" scanner.1 is a perfect proof that a proposed bug will be "active" without a few lockers. Each in some type that use two main variables and typically gets a higher validated shot.

The combined flaw is separated from a battery of UVBs, which naturally infringe on
